In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import regexp_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.corpora as corpora
import gensim
from datetime import datetime, timedelta
import time
import pickle
import pprint

import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
wnl = WordNetLemmatizer()

def func_lemmatize(words):
    lemmatized = []
    for word, tag in pos_tag(words):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a','r','n','v'] else None
        
        lemma = wnl.lemmatize(word,wntag) if wntag else word
        lemmatized.append(lemma)
    return lemmatized

stop = set(stopwords.words('english'))

pattern = r'(\w+)'

### Other Lemmatizing Functions

In [46]:
## def func_lemmatize(words):
##     allow = ['a','r','n','v']
##     
## #    list_1 = [(word,tag[0].lower() if tag[0].lower() in allow else None) for word,tag in pos_tag(words)]
##     list_2 = [wnl.lemmatize(word,wntag) if wntag else word for word, wntag in [(word,tag[0].lower() if tag[0].lower() in allow else None) for word,tag in pos_tag(words)]]
##     
##     return list_2

In [15]:
## def func_lemmatize(words):
##     allow = ['a','r','n','v']
##     
##     list_1 = [(word,tag[0].lower() if tag[0].lower() in allow else None) for word,tag in pos_tag(words)]
##     list_2 = [wnl.lemmatize(word,wntag) if wntag else word for word, wntag in list_1]
##     
##     return list_2

In [ ]:
## def func_lemmatize(words):
##     allow = ['a','r','n','v']
##     
##     lemmatized = [wntag if wntag[0].lower() in allow for word,tag in pog_tag(words)]
##     lemma = [wnl.lemmatize(item) for item in list_1]

### /Other Lemmatizing Functions

In [6]:
# 12:05 (5 mins)
#df = pd.read_csv('all-the-news-2-1.csv')
df= pd.read_csv('../all-the-news-2-1.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication
0,0,0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,1,1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2,2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,3,3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,4,4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [8]:
df = df.dropna(subset = ['article'])

In [9]:
df.reset_index(inplace=True,drop=True)

In [10]:
df.shape

(2584165, 12)

### Select a Number of Records to Use

In [12]:
num = 250000
np.random.seed(3)
inds_to_use = np.random.choice(df.index,num,replace=False)
df2 = df.loc[inds_to_use,:]
#df2 = df.loc[:num]
df2.reset_index(drop=True,inplace=True)

In [13]:
len(df2)

250000

In [19]:
# This function works on summarized versions of articles
## def clean(df):
##     start = time.time()
##     df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
##     df['article_para'] = [article[:120] for article in df['article_words']]
##     df['article_title'] = [regexp_tokenize(title,pattern) for title in df['title']]
##     df['summary'] = [df['article_title'][i] + df['article_para'][i] for i in range(len(df))]
##     
##     df['summary_words'] = [[word for word in article if word.isalpha()] for article in df['summary']]
##     df['summary_words'] = [func_lemmatize(article) for article in df['summary_words']]
##     df['summary_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['summary_words']]
##     interval = round((time.time() - start)/60,2)
##     print(f'That  took {interval} mins.')
##     return df

In [14]:
# Original version
def clean(df):
    start = time.time()
    df['article_words'] = [regexp_tokenize(article,pattern) for article in df['article']]
    print('tokenizing done.')  
    df['article_words'] = [[word for word in article if word.isalpha()] for article in df['article_words']]
    print('isalpha done.')
    df['article_words'] = [func_lemmatize(article) for article in df['article_words']]
    print('lemmatizing done.')
    df['article_words'] = [[word.lower() for word in article if word.lower() not in stop] for article in df['article_words']]
    interval = round((time.time() - start)/60,2)
    print(f'That  took {interval} mins.')
    return df

In [15]:
# Original Function
# 09:24
df2 = clean(df2)

tokenizing done.
isalpha done.
lemmatizing done.
That  took 133.46 mins.


In [16]:
df2.head()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section,publication,article_words
0,539903,539903,2017-06-01 00:00:00,2017,6.0,1,James Vincent,Misfit’s new tool lets you customize your fitn...,Misfit has launched a new tool that lets users...,https://www.theverge.com/circuitbreaker/2017/6...,Tech,The Verge,"[misfit, launch, new, tool, let, user, persona..."
1,255873,255873,2019-07-11 00:00:00,2019,7.0,11,Marcelo Teixeira,"Coffee farmers to form legal entity, launch pl...","CAMPINAS, Brazil, July 11 (Reuters) - Particip...",https://www.reuters.com/article/coffee-forum-o...,Non-Cyclical Consumer Goods,Reuters,"[campinas, brazil, july, reuters, participants..."
2,2403288,2415652,2019-11-06 00:00:00,2019,11.0,6,NaN,China supports more aggressive measures to tam...,"BEIJING, Nov 6 (Reuters) - China’s Vice Premie...",https://www.reuters.com/article/hongkong-prote...,Cyclical Consumer Goods,Reuters,"[beijing, nov, reuters, china, vice, premier, ..."
3,1328201,1328201,2017-01-11,2017,1.0,11,NaN,BRIEF-Selz Capital LLC reports 7.1 pct stake i...,Jan 11 (Reuters) - WPX Energy Inc * Selz Capit...,http://www.reuters.com/article/idUSFWN1F10JT,Funds News,Reuters,"[jan, reuters, wpx, energy, inc, selz, capital..."
4,2069896,2069896,2017-11-29 22:00:01,2017,11.0,29,Deb Amlen,Sinking It,Wordplay THURSDAY PUZZLE — Today’s puzzle by T...,https://www.nytimes.com/2017/11/29/crosswords/...,crosswords,The New York Times,"[wordplay, thursday, puzzle, today, puzzle, tr..."


In [18]:
# Save to csv
df2.to_csv('250k_full.csv')